In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder
from scipy import stats
%matplotlib inline

pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows',500)


In [2]:
df = pd.read_csv('US_births(2018).csv')


C:\Users\saden\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.head()

,ATTEND,BFACIL,BMI,CIG_0,DBWT,DLMP_MM,DLMP_YY,DMAR,DOB_MM,DOB_TT,DOB_WK,DOB_YY,DWgt_R,FAGECOMB,FEDUC,FHISPX,FRACE15,FRACE31,FRACE6,ILLB_R,ILOP_R,ILP_R,IMP_SEX,IP_GON,LD_INDL,MAGER,MAGE_IMPFLG,MAR_IMP,MBSTATE_REC,MEDUC,MHISPX,MM_AICU,MRACE15,MRACE31,MRACEIMP,MRAVE6,MTRAN,M_Ht_In,NO_INFEC,NO_MMORB,NO_RISKS,PAY,PAY_REC,PRECARE,PREVIS,PRIORDEAD,PRIORLIVE,PRIORTERM,PWgt_R,RDMETH_REC,RESTATUS,RF_CESAR,RF_CESARN,SEX,WTGAIN
0,1,1,30.7,0,3657,4,2017,1,1,1227,2,2018,231,31,3,1,1,1,1,16,33,16,,N,N,30,,,1,6,0,N,1,1,,1,N,66,1,1,1,2,2,3,8,0,1,2,190,1,2,N,0,M,41
1,1,1,33.3,2,3242,99,9999,2,1,1704,2,2018,185,35,4,0,3,3,3,180,888,180,,N,N,35,,,1,9,0,N,3,3,,3,N,63,1,1,0,1,1,3,9,0,2,0,188,4,2,Y,2,F,0
2,1,1,30.0,0,3470,4,2017,1,1,336,2,2018,273,31,4,0,1,1,1,999,888,999,,N,N,28,,,1,6,0,N,1,1,,1,N,71,1,1,0,5,4,5,17,0,1,0,215,1,1,N,0,M,58
3,3,1,23.7,0,3140,5,2017,2,1,938,2,2018,138,26,2,0,3,3,3,43,888,43,,N,N,23,,,1,2,0,N,3,3,,3,N,64,1,1,1,1,1,5,6,0,2,0,138,1,2,N,0,F,0
4,1,1,35.5,0,2125,99,9999,1,1,830,3,2018,219,35,3,0,2,2,2,999,999,999,,N,N,37,,,1,4,0,N,1,1,,1,N,66,1,1,1,1,1,5,15,0,1,4,220,3,1,N,0,M,0


In [4]:
df.shape

(3801534, 55)

In [5]:
def drop_rows(df):
    '''
    Dropping rows where missing: 'DBWT', 'BMI', 'DBWT', 'WTGAIN', 'PWgt_R', 'DLMP_MM', 'DLMP_YY'
    '''
    df.drop(df[df['DBWT'].eq(9999)].index, inplace=True)    
    df.drop(df[df['BMI'].eq(99.9)].index, inplace=True)
    df.drop(df[df['DBWT'].eq(9999)].index, inplace=True)
    df.drop(df[df['WTGAIN'].eq(99)].index, inplace=True)
    df.drop(df[df['PWgt_R'].eq(999)].index, inplace=True)
    df.drop(df[df['DLMP_MM'].eq(99)].index, inplace=True)
    df.drop(df[df['DLMP_YY'].eq(9999)].index, inplace=True)
    
    df.drop(columns=['IMP_SEX'], inplace=True)
    return df
df = drop_rows(df)

In [6]:
df.shape

(3521224, 54)

In [7]:
df.head()

,ATTEND,BFACIL,BMI,CIG_0,DBWT,DLMP_MM,DLMP_YY,DMAR,DOB_MM,DOB_TT,DOB_WK,DOB_YY,DWgt_R,FAGECOMB,FEDUC,FHISPX,FRACE15,FRACE31,FRACE6,ILLB_R,ILOP_R,ILP_R,IP_GON,LD_INDL,MAGER,MAGE_IMPFLG,MAR_IMP,MBSTATE_REC,MEDUC,MHISPX,MM_AICU,MRACE15,MRACE31,MRACEIMP,MRAVE6,MTRAN,M_Ht_In,NO_INFEC,NO_MMORB,NO_RISKS,PAY,PAY_REC,PRECARE,PREVIS,PRIORDEAD,PRIORLIVE,PRIORTERM,PWgt_R,RDMETH_REC,RESTATUS,RF_CESAR,RF_CESARN,SEX,WTGAIN
0,1,1,30.7,0,3657,4,2017,1,1,1227,2,2018,231,31,3,1,1,1,1,16,33,16,N,N,30,,,1,6,0,N,1,1,,1,N,66,1,1,1,2,2,3,8,0,1,2,190,1,2,N,0,M,41
2,1,1,30.0,0,3470,4,2017,1,1,336,2,2018,273,31,4,0,1,1,1,999,888,999,N,N,28,,,1,6,0,N,1,1,,1,N,71,1,1,0,5,4,5,17,0,1,0,215,1,1,N,0,M,58
3,3,1,23.7,0,3140,5,2017,2,1,938,2,2018,138,26,2,0,3,3,3,43,888,43,N,N,23,,,1,2,0,N,3,3,,3,N,64,1,1,1,1,1,5,6,0,2,0,138,1,2,N,0,F,0
5,4,2,31.3,0,4082,3,2017,1,1,28,2,2018,247,28,6,6,1,1,1,39,888,39,N,N,26,,,1,6,0,N,1,1,,1,N,67,1,1,1,2,2,2,13,0,1,0,200,1,1,N,0,F,47
6,1,1,23.0,0,3180,3,2017,1,1,341,3,2018,170,25,6,0,1,1,1,27,888,27,N,Y,28,,,1,4,0,N,1,1,,1,N,70,1,1,1,5,4,3,6,0,2,0,160,1,1,N,0,F,10


In [8]:
def feature_engineer(df):
    '''
    Creating new column: 'first_birth', 'smoked', 'PRIORDEAD_cat', 'PRIORTERM_cat', 'PRIORLIVE_cat', 
    
    '''

    #creating new column called 'first_birth': Is the baby the Mom's first child? Yes:1 No:0
    df['first_birth'] = np.where(df['ILLB_R'].eq(888), 1, 0)
    df['plural_delivery'] = np.where(df['ILLB_R'].lt(4), 'Yes', 'No')


    
    #creating new column 'smoked': Did the mother smoke before pregnancy? Yes: Daily, No: None, Unknown: Unknown
    conditions = [df['CIG_0'].eq(0),
                  df['CIG_0'].eq(99)]
    choices = ['None',
               'Unknown']
    df['smoked'] = np.select(conditions, choices, 'Daily')

    
    #creating new column 'PRIORDEAD_cat': Did the mother previously have miscarriages? Yes: Yes, No: None, Unknown: Unknown
    conditions = [df['PRIORDEAD'].eq(0),
                  df['PRIORDEAD'].eq(99)]
    choices = ['None',
               'Unknown']
    df['PRIORDEAD_cat'] = np.select(conditions, choices, 'Yes')

    
    #creating new column 'PRIORTERM_cat': Did the mother previously have terminations? Yes: Yes, No:None, Unknown: Unknown
    conditions = [df['PRIORTERM'].eq(0),
                  df['PRIORTERM'].eq(99)]
    choices = ['None',
               'Unknown']
    df['PRIORTERM_cat'] = np.select(conditions, choices, 'Yes')

    #creating new column 'PRIORLIVE_cat': Did the mother previously birth living children: Yes: Yes, No: None, Unknown: Unknown
    conditions = [df['PRIORLIVE'].eq(0),
                  df['PRIORLIVE'].eq(99)]
    choices = ['None',
               'Unknown']
    df['PRIORLIVE_cat'] = np.select(conditions, choices, 'Yes')

    # creating new column 'pregnancy_length': An estimation of the gestation period by subtracting the month/year of last menses from month/year of baby born
    conditions = [(df['DOB_MM'] > df['DLMP_MM']) & (2018 == df['DLMP_YY']),
                  (df['DOB_MM'] > df['DLMP_MM']) & (2018 > df['DLMP_YY']),
                  (df['DOB_MM'] < df['DLMP_MM']) & (2018 > df['DLMP_YY'])]
    choices = [df['DOB_MM'] - df['DLMP_MM'],
               ((df['DOB_YY'] - df['DLMP_YY'])* 12) + df['DOB_MM'] - df['DLMP_MM'],
               ((df['DOB_YY'] - df['DLMP_YY'])* 12) - df['DLMP_MM'] + df['DOB_MM']]
    df['pregnancy_length'] = np.select(conditions, choices, 12)
    
    #creating new column 'MAGER_cat': Mother's age is < 18: Minor, Mother's age is >= 18: Adult
    df['MAGER_cat'] = np.where(df['MAGER'].lt(18),'Minor','Adult')

    #creating new column 'pregnancy_cat: Binning pregnancy_length to 5 bins: 'Early','8','9','10','Late'
    condition =[df['pregnancy_length'].eq(9),
                df['pregnancy_length'].eq(8),
                df['pregnancy_length'].eq(10),
                df['pregnancy_length'].lt(7)]
    choices = ['9',
               '8',
               '10',
               'Early']
    df['pregnancy_length_cat'] = np.select(condition, choices, 'Late')

    #creating new column 'BMI_log':np.log(BMI) to normalize BMI
    df['BMI_log'] = np.log(df['BMI'])
    
    #creating new column 'first_pregnancy': Is it mother's first live birth? Yes: 1, No: 0
    df['first_live_birth'] = np.where(df['ILP_R'].eq(888), 1, 0)
    
    #creating new column 'first_natal': Is it mother's first natality event? Yes: 1, No: 0
    df['first_natal'] = np.where(df['ILOP_R'].eq(888),1, 0)
    
    #adjusting 'PRECARE' values: if Unkonwn, impute 0
    df['PRECARE'] = np.where(df['PRECARE'].eq(99), 0, df['PRECARE'])
    
    #adjusting 'PREVIS' values: if Unkonwn, impute 0
    df['PREVIS'] = np.where(df['PREVIS'].eq(99), 0, df['PREVIS'])
    
    #creating new column 'T35AGE_older': Is mother's age older than 34? Yes: 1, No: 0
    df['T35AGE_older'] = np.where(df['MAGER'].gt(34), 1, 0)
    
    #creating new column 'MOM_weight': manually computing mom's weight incase of missing values
    df['MOM_weight'] = (df['M_Ht_In']**2)*df['BMI'] / 704
    
    #creating new column '...': calculating percentage of weight gained due to pregnancy
    df['WT_percent_gain'] = df['WTGAIN'] / df['PWgt_R']
    
    #adjusting 'MAR_IMP': Marriage imputed should be 0 if left blank
    df['MAR_IMP'] = np.where(df['MAR_IMP'].eq(' '), 0, 1)
    
    #adjusting 'DMAR': assigning blanks to a new variable 0 for unknowns
    df['DMAR'] = np.where(df['DMAR'].eq(' '), 0, df['DMAR'])
    
    # #creating new column '...':
    # df['pregnancy_length_sqrt'] =  np.sqrt(df['pregnancy_length'])
    
    #dropping rows that with gestation period greater than 12 and less than 5, treating them as outliers
    df.drop(df[df['pregnancy_length'].gt(12)].index, inplace=True)
    df.drop(df[df['pregnancy_length'].lt(5)].index, inplace=True)
    return df

df = feature_engineer(df)
df.reset_index(inplace=True, drop=True)

In [9]:
df.shape

(3511812, 69)

In [10]:
df['DBWT_cat'] = np.where(df['DBWT'].lt(2500), 'Low', 'Normal')

In [11]:
df.shape

(3511812, 70)

In [12]:
df['DBWT_ca'] = np.where(df['DBWT'].lt(2500), 1, 2)

In [13]:
df.shape

(3511812, 71)

In [22]:
    #creating new column 'smoked': Did the mother smoke before pregnancy? Yes: 1, No: 2, Unknown: 3
    conditions = [df['CIG_0'].eq(0),
                  df['CIG_0'].eq(99)]
    choices = [2,
               3]
    df['Smoked_Int'] = np.select(conditions, choices, 1)

In [23]:
df.shape

(3511812, 73)

In [24]:
df['Smoked_Int'].value_counts()

2    3217052
1     282349
3      12411
Name: Smoked_Int, dtype: int64

In [17]:
df['DBWT_cat'].value_counts()

Normal    3233101
Low        278711
Name: DBWT_cat, dtype: int64

In [25]:
df.to_csv('US_Births_processed.csv', index=False)

In [18]:
df = df.dropna()

In [19]:
df.shape

(3511812, 72)

In [20]:
pd.isnull(df)

df.isnull().sum()

ATTEND                  0
BFACIL                  0
BMI                     0
CIG_0                   0
DBWT                    0
DLMP_MM                 0
DLMP_YY                 0
DMAR                    0
DOB_MM                  0
DOB_TT                  0
DOB_WK                  0
DOB_YY                  0
DWgt_R                  0
FAGECOMB                0
FEDUC                   0
FHISPX                  0
FRACE15                 0
FRACE31                 0
FRACE6                  0
ILLB_R                  0
ILOP_R                  0
ILP_R                   0
IP_GON                  0
LD_INDL                 0
MAGER                   0
MAGE_IMPFLG             0
MAR_IMP                 0
MBSTATE_REC             0
MEDUC                   0
MHISPX                  0
MM_AICU                 0
MRACE15                 0
MRACE31                 0
MRACEIMP                0
MRAVE6                  0
MTRAN                   0
M_Ht_In                 0
NO_INFEC                0
NO_MMORB    

In [21]:
df.to_csv('US_Births_processed.csv', index=False)

In [37]:
df['CIG_0']

0          0
1          0
2          0
3          0
4          0
          ..
3511807    0
3511808    0
3511809    0
3511810    0
3511811    0
Name: CIG_0, Length: 3511812, dtype: int64

# ** STOP ** my future work 

In [ ]:
X = df[['ATTEND','BFACIL', 'smoked', 'DOB_MM', 'DMAR','FHISPX','FEDUC', 'FRACE6', 'first_birth', 'plural_delivery', 'first_live_birth', 'first_natal',
        'IP_GON', 'LD_INDL', 'MAGER', 'T35AGE_older','MAR_IMP', 'MBSTATE_REC', 'MEDUC', 'MHISPX', 'MRAVE6', 'MTRAN', 'pregnancy_length',
        'NO_INFEC','NO_MMORB','NO_RISKS','PAY', 'PAY_REC','PRECARE','PREVIS', 'PRIORDEAD_cat', 'PRIORLIVE_cat', 'PRIORTERM_cat', 'PWgt_R', 'BMI_log','M_Ht_In', 'MOM_weight',
        'RDMETH_REC', 'RESTATUS', 'RF_CESAR', 'SEX', 'WTGAIN','WT_percent_gain','MAGER_cat','pregnancy_length_cat','BMI'
]]

_X = pd.get_dummies(X, columns=['ATTEND', 'BFACIL','smoked', 'DOB_MM','DMAR','FHISPX','FEDUC','FRACE6', 'plural_delivery',
                                'IP_GON','LD_INDL', 'T35AGE_older','MAR_IMP', 'MBSTATE_REC', 'MEDUC', 'MHISPX', 'MRAVE6', 'MTRAN',
                                'NO_INFEC','NO_MMORB','NO_RISKS', 'PAY', 'PAY_REC','PRIORDEAD_cat', 'PRIORLIVE_cat','PRIORTERM_cat',
                                'RDMETH_REC', 'RESTATUS', 'RF_CESAR', 'SEX','MAGER_cat','pregnancy_length_cat']).copy()
y = df['DBWT']
_X.shape

In [ ]:
# creating a list of continuous columns will help us when we want to scale only our continuous variables

_X_columns = _X.columns
categorical_columns = []
continuous_columns = []
for i in _X_columns:
    if _X[i].max() == 1:
        categorical_columns.append(i)
    else:
        continuous_columns.append(i)s
        
del _X_columns

In [ ]:
_X['DBWT'] = y
plt.figure(figsize=(12,10))
sns.distplot(_X['DBWT'])
plt.title('Distribution plot of Baby Weight in Grams')
plt.xlabel('Baby Weight in Grams')
plt.savefig('figure1.png')

In [ ]:
_X['DBWT'] = y
from statsmodels.formula.api import ols
formula = 'DBWT~'+'+'.join(_new_columns)
model = ols(formula=formula, data=_X).fit()
model.summary()